In [3]:
import tonic
import tonic.transforms as transforms
import numpy as np
import tensorflow as tf

transform = transforms.ToFrame(sensor_size = (1280, 720), time_window = 100000)


In [33]:
path = "/Volumes/CVI2_SPACE_1/SPADES"


In [36]:
import tonic
import os
import pandas as pd
import numpy as np

class SpacecraftEventDataset(tonic.Dataset):
    def __init__(self, root_dir, dataset_type="synthetic", transform=None):
        """
        Args:
            root_dir (str): Root directory containing the synthetic or Real dataset folders.
            dataset_type (str): Type of dataset to load ("Synthetic" or "Real").
            transform (callable, optional): Optional transform to apply to the events.
        """
    
        self.root_dir = root_dir
        self.dataset_type = dataset_type
        self.samples = self._load_samples()

    def _load_samples(self):
        """
        Load event and label files and pair them appropriately.
        
        Returns:
            list of tuples: Each tuple contains paths to an event file and its corresponding label file (for synthetic data).
        """
        samples = []
        events_dir = os.path.join(self.root_dir, self.dataset_type, "events")
        
        if self.dataset_type == "synthetic":
            labels_dir = os.path.join(self.root_dir, self.dataset_type, "labels")
            for file_name in sorted(os.listdir(events_dir)):
                if file_name.endswith(".csv"):
                    event_file = os.path.join(events_dir, file_name)
                    label_file = os.path.join(labels_dir, file_name)
                    samples.append((event_file, label_file))
        elif self.dataset_type == "Real":
            for file_name in sorted(os.listdir(events_dir)):
                if file_name.endswith(".csv"):
                    event_file = os.path.join(events_dir, file_name)
                    samples.append((event_file, None))
        return samples

    def _load_events(self, file_path, is_synthetic=True):
        """
        Load event data from a CSV file.
        
        Args:
            file_path (str): Path to the event CSV file.
            is_synthetic (bool): Whether the event data is synthetic or real.

        Returns:
            np.ndarray: Array of events with columns [timestamp, x, y, polarity].
        """
        df = pd.read_csv(file_path)
        events = df[['timestamp', 'x', 'y', 'polarity']].values
        return events

    def _load_labels(self, file_path):
        """
        Load label data from a CSV file.
        
        Args:
            file_path (str): Path to the label CSV file.

        Returns:
            np.ndarray: Array of labels with columns [Tx, Ty, Tz, Qx, Qy, Qz, Qw].
        """
        df = pd.read_csv(file_path)
        labels = df[['Tx', 'Ty', 'Tz', 'Qx', 'Qy', 'Qz', 'Qw']].values
        return labels

    def __getitem__(self, index):
        """
        Get a sample and its label (if available).

        Args:
            index (int): Index of the sample.

        Returns:
            tuple: (events, labels) where events is an array of event data, and labels is an array of labels (if synthetic data).
        """
        event_file, label_file = self.samples[index]
        is_synthetic = self.dataset_type == "synthetic"
        
        events = self._load_events(event_file, is_synthetic=is_synthetic)
        if label_file:
            labels = self._load_labels(label_file)
        else:
            labels = None

        if self.transform:
            events = self.transform(events)

        return events, labels

    def __len__(self):
        return len(self.samples)

In [37]:
dataset = SpacecraftEventDataset(root_dir=path, transform=transform)

OSError: [Errno 22] Invalid argument: '/Volumes/CVI2_SPACE_1/SPADES/synthetic/events'